In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf
import torch
%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/network/vit_lines.yaml"

In [38]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [39]:
cfg

{'_target_': 'text_recognizer.network.vit.VisionTransformer', 'image_height': 56, 'image_width': 1024, 'patch_height': 28, 'patch_width': 32, 'dim': 256, 'num_classes': 57, 'encoder': {'_target_': 'text_recognizer.network.transformer.encoder.Encoder', 'dim': 256, 'inner_dim': 1024, 'heads': 8, 'dim_head': 64, 'depth': 6, 'dropout_rate': 0.0}, 'decoder': {'_target_': 'text_recognizer.network.transformer.decoder.Decoder', 'dim': 256, 'inner_dim': 1024, 'heads': 8, 'dim_head': 64, 'depth': 6, 'dropout_rate': 0.0}, 'token_embedding': {'_target_': 'text_recognizer.network.transformer.embedding.token.TokenEmbedding', 'num_tokens': 57, 'dim': 256, 'use_l2': True}, 'pos_embedding': {'_target_': 'text_recognizer.network.transformer.embedding.absolute.AbsolutePositionalEmbedding', 'dim': 256, 'max_length': 89, 'use_l2': True}, 'tie_embeddings': True, 'pad_index': 3}

In [40]:
net = instantiate(cfg)

In [41]:
from torchinfo import summary

In [43]:
summary(net, ((1, 1, 56, 1024), (1, 89)), device="cpu", depth=4)

Layer (type:depth-idx)                             Output Shape              Param #
VisionTransformer                                  [1, 57, 89]               --
├─Sequential: 1-1                                  [1, 64, 256]              --
│    └─Rearrange: 2-1                              [1, 64, 896]              --
│    └─LayerNorm: 2-2                              [1, 64, 896]              1,792
│    └─Linear: 2-3                                 [1, 64, 256]              229,632
│    └─LayerNorm: 2-4                              [1, 64, 256]              512
├─Encoder: 1-2                                     [1, 64, 256]              --
│    └─ModuleList: 2-5                             --                        --
│    │    └─ModuleList: 3-1                        --                        --
│    │    │    └─Attention: 4-1                    [1, 64, 256]              525,824
│    │    │    └─FeedForward: 4-2                  [1, 64, 256]              526,080
│    │    └─Modu

In [20]:
import torch

In [24]:
t = net(torch.randn(1, 1, 56, 1024), torch.randint(1, 4, (1, 4)))

In [25]:
t.shape

torch.Size([1, 58, 4])

In [17]:
loss = torch.nn.CrossEntropyLoss()

In [19]:
loss(t.permute(0, 2, 1), torch.randint(0, 58, (1, 89)))

tensor(4.0604, grad_fn=<NllLoss2DBackward0>)